# Network Analysis

I conducted a network analysis at work to understand how internal connections can help a staff progress in his/her career. I used NetworkX package, and I think it is extremely useful. I am sharing here codes to create an interactive network graph, combining Plotly and NetworkX, and some insights to analyze a network. 
  
There are 2 datasets provided: one contains names and team ID, and the other has a list of unique names with genders and departments. 


Disclaimer: this is not the real data.

In [1]:
import plotly.plotly as py
from plotly.graph_objs import *
import networkx as nx
import pandas as pd
import numpy as np
from networkx.algorithms import bipartite

from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)

### Load and prepare data for NetworkX

In [2]:
team_data = pd.read_excel('Sample_team_dataset.xlsx')
staff_attributes = pd.read_excel('Sample_team_member_detail.xlsx')

staff_attributes = staff_attributes.set_index('Name')
team_data['edges']=list(zip(team_data['Team ID'], team_data.Name))

In [3]:
team_data.head(5)

,Team ID,Name,edges
0,0,Hayden,"(0, Hayden)"
1,0,Jack,"(0, Jack)"
2,0,Jaden,"(0, Jaden)"
3,0,Ella,"(0, Ella)"
4,0,Khloe,"(0, Khloe)"


In [4]:
staff_attributes.head(5)

,Gender,Department
Name,,
Mason,M,10
William,M,10
Olivia,F,10
Noah,M,4
Michael,M,10


### Create network graph
The assumption for connections is that all employees in the same team are connected. Therefore, a bipartite graph, consisting of one set of team id and one set of staff names, would fit this assumption. NetworkX already provides algorithms to create and analyze bipartite graphs.  

In [5]:
#Create lists of nodes and edges 
names = list(set(team_data['Name']))
teamid = list(set(team_data['Team ID']))
team_edges = list(team_data['edges'])

In [6]:
G = nx.Graph()
G.add_nodes_from(names, bipartite=0)
G.add_nodes_from(teamid, bipartite = 1)
G.add_edges_from(team_edges)

In [7]:
#Check if G is bipartite
bipartite.is_bipartite(G)

True

Now I want to create a projected graph to analyze network of employees who have at least a team in common.  

In [8]:
P = bipartite.weighted_projected_graph(G, set(names))

In [9]:
print ('Number of nodes: ',P.number_of_nodes(),'\nNumber of edges: ',P.number_of_edges())

Number of nodes:  153 
Number of edges:  871


In [10]:
#Check if graph P is connected
nx.is_connected(P)

True

You can create a quick network graph using NetworkX. However, with a large number of nodes and edges (as in my case at work), I found graphs generated by NetworkX not intuitive, so I decided to combine Plotly and NetworkX to create an interactive chart with more vibrant colors.  

In [11]:
def graph_network(G, nodes_attr_df, title):    
    pos = nx.spring_layout(G)
    for n in G:
        G.node[n]['pos']=pos[n]
        G.node[n]['gender']=nodes_attr_df.loc[n, 'Gender']
        
    
    #Add edges as disconnected lines in a single trace and nodes as a scatter trace
    edge_trace = Scatter(
        x=[],
        y=[],
        line=Line(width=0.1,color='#888'),
        hoverinfo='none',
        mode='lines')

    for edge in G.edges():
        x0, y0 = G.node[edge[0]]['pos']
        x1, y1 = G.node[edge[1]]['pos']
        edge_trace['x'] += [x0, x1, None]
        edge_trace['y'] += [y0, y1, None]
    
    #create different colors for a categorical attribute of nodes. Here I used gender as an example
    node_trace1 = Scatter(
    x=[],
    y=[],
    text=[],
    mode='markers',
    hoverinfo='text',
    marker=Marker(
        showscale=True,
            # colorscale options
            # 'Greys' | 'Greens' | 'Bluered' | 'Hot' | 'Picnic' | 'Portland' |
            # Jet' | 'RdBu' | 'Blackbody' | 'Earth' | 'Electric' | 'YIOrRd' | 'YIGnBu'
        colorscale='YIGnBu',
        reversescale=True,
        color=[],
        size=10,
        colorbar=dict(
            thickness=15,
            title='Node Connections (Male scale)',
            xanchor='left',
            titleside='right'
            ),
        line=dict(width=2)))

    node_trace2 = Scatter(
        x=[],
        y=[],
        text=[],
        mode='markers',
        hoverinfo='text',
        marker=Marker(
            showscale=True,
                # colorscale options
                # 'Greys' | 'Greens' | 'Bluered' | 'Hot' | 'Picnic' | 'Portland' |
                # Jet' | 'RdBu' | 'Blackbody' | 'Earth' | 'Electric' | 'YIOrRd' | 'YIGnBu'
            colorscale='YIOrRd',
            reversescale=True,
            color=[],
            size=10,
            colorbar=dict(
                thickness=15,
                title='Node Connections (Female scale)',
                x=-0.05,
                titleside='right'
                ),
            line=dict(width=2)))

    for node in G.nodes():
        if G.node[node]['gender']=='M':
            x, y = G.node[node]['pos']
            node_trace1['x'].append(x)
            node_trace1['y'].append(y)
        else:
            x, y = G.node[node]['pos']
            node_trace2['x'].append(x)
            node_trace2['y'].append(y)
    
    #Color node points by the number of connections.
    for node, adjacencies in G.adjacency_iter():
        if G.node[node]['gender']=='M':
            node_trace1['marker']['color'].append(len(adjacencies))
            node_info = str(node) + ' - # of connections: '+str(len(adjacencies))
            node_trace1['text'].append(node_info)
        else:
            node_trace2['marker']['color'].append(len(adjacencies))
            node_info = str(node)+' - # of connections: '+str(len(adjacencies))
            node_trace2['text'].append(node_info)
        
    #Create Network Graph
    fig = Figure(data=Data([edge_trace, node_trace1, node_trace2]),
                 layout=Layout(
                    title='<br>'+title,
                    titlefont=dict(size=16),
                    showlegend=False,
                    hovermode='closest',
                    margin=dict(b=20,l=5,r=5,t=40),
                    xaxis=XAxis(showgrid=False, zeroline=False, showticklabels=False),
                    yaxis=YAxis(showgrid=False, zeroline=False, showticklabels=False)))

    return fig

In [12]:
title = 'Network Graph'
fig = graph_network(P, staff_attributes,title )
iplot(fig)

Here is the link of this interactive chart: 
https://plot.ly/~hailey_n5/153/

### Analyze the staff network

##### First, let's examine some nodes' attributes and edges' attributes

In [13]:
#Load nodes' attributes from 'Sample_team_member_detail.xlsx' (staff_attributes dataframe) into graph P
for n in P:
    P.node[n]['department']=staff_attributes.loc[n, 'Department']
    P.node[n]['gender']=staff_attributes.loc[n, 'Gender']

In [14]:
P.nodes(data=True)[:5]

[('Reese',
  {'bipartite': 0,
   'department': 8,
   'gender': 'F',
   'pos': array([0.06144496, 0.69251423])}),
 ('London',
  {'bipartite': 0,
   'department': 7,
   'gender': 'F',
   'pos': array([0.65785471, 0.51446434])}),
 ('Angel',
  {'bipartite': 0,
   'department': 1,
   'gender': 'M',
   'pos': array([0.57049213, 0.05488789])}),
 ('Cameron',
  {'bipartite': 0,
   'department': 8,
   'gender': 'M',
   'pos': array([0.60212786, 0.0465941 ])}),
 ('Richard',
  {'bipartite': 0,
   'department': 8,
   'gender': 'M',
   'pos': array([0.6302998 , 0.38057685])})]

In [15]:
P.edges(data=True)[:10] #weight is the number of teams these two nodes have in common

[('Reese', 'Victor', {'weight': 1}),
 ('Reese', 'Grant', {'weight': 1}),
 ('Reese', 'Elias', {'weight': 1}),
 ('Reese', 'Brielle', {'weight': 1}),
 ('Reese', 'Nora', {'weight': 1}),
 ('Reese', 'Jesus', {'weight': 1}),
 ('Reese', 'Ivan', {'weight': 1}),
 ('London', 'Santiago', {'weight': 1}),
 ('London', 'Jonah', {'weight': 1}),
 ('London', 'Nicole', {'weight': 1})]

##### Next, I would like to create some node based features that I found useful to understand a network and to fit a model

1.Local clustering coefficient: 


    Local clustering coefficient of node A = 
    (# of pairs of A's friends who are friends) / (# of pairs of A's friends)

   
2.Degree centrality:

    This measure focuses on the number of connections
    
    degCent(u) = (degree of node u) / (# of nodes in the network - 1)
    
    
3.Closeness centrality:

    This measure focuses on the distance between a node and other nodes
    
    CloseCent(u) = 
    (# of nodes in the network - 1) / (Sum of lengths of shortest path from u to other nodes)
   
$$C(u) = \frac{n - 1}{\sum_{v=1}^{n-1} d(v, u)},$$
  
  
4.Betweenness centrality:

    This measure emphasizes nodes that act as bridges between other nodes
    
    BtwCent(u) = 
    Sum of ((# of shortest paths between nodes s & t that pass through node u) / 
             (# of shortest paths between nodes s & t))
    s, t are in the set of nodes in the network
    
$$c_B(u) =\sum_{s,t \in V} \frac{\sigma(s, t|u)}{\sigma(s, t)}$$
    
    For graphs that have many nodes, I would recommend normalizing this measure by dividing it by:
    (1/2)*(# of nodes in the network - 1)*(# of nodes in the network - 2)  for undirected graphs
    (# of nodes in the network - 1)*(# of nodes in the network - 2) for directed graphs

In [16]:
#Local clustering coefficient:
staff_attributes['Clustering'] = pd.Series(nx.clustering(P))

#Degree centrality:
staff_attributes['DegCent'] = pd.Series(nx.degree_centrality(P))

#Closeness centrality:
staff_attributes['CloseCent'] = pd.Series(nx.closeness_centrality(P))

#Betweenness centrality:
staff_attributes['BtwCent'] = pd.Series(nx.betweenness_centrality(P,normalized = True, endpoints = False))

In [17]:
staff_attributes.head(10)

,Gender,Department,Clustering,DegCent,CloseCent,BtwCent
Name,,,,,,
Mason,M,10,1.000000,0.026316,0.278388,0.000000
William,M,10,1.000000,0.026316,0.278388,0.000000
Olivia,F,10,1.000000,0.026316,0.278388,0.000000
Noah,M,4,1.000000,0.026316,0.278388,0.000000
Michael,M,10,1.000000,0.026316,0.177156,0.000000
Ethan,M,10,0.733333,0.039474,0.196382,0.004830
Alexander,M,10,0.733333,0.039474,0.196382,0.004830
Aiden,M,10,0.555556,0.059211,0.214085,0.018909
Chloe,F,10,1.000000,0.026316,0.177156,0.000000


##### Edge based features are important in predicting future connections of non-existent edges in the graph

The most common measures for edges are common neighbors and preferential attachment. When predicting future connections of currently non-existent edges, I prefer adding community common neighbors measure with an assumption that employees in the same department (community) will be more likely to form a connection in the future than those in different departments.

1.Common neighbors:
    
    = Number of common neighbors between 2 nodes
    
    With a large number of nodes, I would want to normalize this measure by the total number of neighbors 
    (Jaccard Coefficient)

2.Preferential attachment:
    
    = Product of the nodes' degrees
    
    This is based on the preferential attachment model where nodes with high degrees get more neighbors.


3.Community common neighbors
    
    = Number of common neighbors of nodes u and v 
    + bonus 1 for each common neighbor belonging to the same community as u and v (f(w))
$$|N(u) \cap N(v)| + \sum_{w \in N(u) \cap N(v)} f(w)$$
    
        N(u) = set of neighbors of u

In [18]:
#Initialize a dataframe for edges with edges as index:
edges_attributes = pd.DataFrame(index=P.edges())

#Add weight to the dataframe:
edges_attributes['Weight'] = pd.Series(nx.get_edge_attributes(P, 'weight'))

#Compute common neighbors measure:
edges_attributes['Comm_neigh'] = edges_attributes.index.map(lambda i: len(list(nx.common_neighbors(P, i[0], i[1]))))

#Compute Jaccard coefficient:
edges_attributes['Jacc_coef'] = [i[2] for i in nx.jaccard_coefficient(P, edges_attributes.index)]

#Compute preferential attachment measure:
edges_attributes['Pref_attachment'] = [i[2] for i in nx.preferential_attachment(P, edges_attributes.index)]


In [19]:
edges_attributes.head()

,Weight,Comm_neigh,Jacc_coef,Pref_attachment
"(Reese, Victor)",1,6,0.750000,49
"(Reese, Grant)",1,6,0.333333,119
"(Reese, Elias)",1,6,0.333333,119
"(Reese, Brielle)",1,6,0.750000,49
"(Reese, Nora)",1,6,0.333333,119


###### Compute features for non-existent edges in the network

In [20]:
#Initialize a dataframe for nonedges with nonedges as index:
nonedges_attr = pd.DataFrame(index=nx.non_edges(P))

len(nonedges_attr)

10757

In [21]:
#Compute community common neighbor measure, using department as community:
nonedges_attr['community_neighbors']=[i[2] for i in nx.cn_soundarajan_hopcroft(P, 
                                                ebunch = nonedges_attr.index,
                                                community = 'department')]

#Compute common neighbors measure:
nonedges_attr['Comm_neigh'] = nonedges_attr.index.map(lambda i: len(list(nx.common_neighbors(P, i[0], i[1]))))

#Compute Jaccard coefficient:
nonedges_attr['Jacc_coef'] = [i[2] for i in nx.jaccard_coefficient(P, nonedges_attr.index)]

#Compute preferential attachment measure:
nonedges_attr['Pref_attachment'] = [i[2] for i in nx.preferential_attachment(P, nonedges_attr.index)]

In [22]:
nonedges_attr.head(10)

,community_neighbors,Comm_neigh,Jacc_coef,Pref_attachment
"(Angel, Timothy)",0,0,0.0,60
"(Angel, Noah)",0,0,0.0,20
"(Angel, Kyle)",0,0,0.0,95
"(Angel, Isaiah)",0,0,0.0,20
"(Angel, Jaxon)",0,0,0.0,45
"(Angel, Levi)",0,0,0.0,70
"(Angel, David)",0,0,0.0,45
"(Angel, Jaxson)",0,0,0.0,60
"(Angel, Nathan)",0,0,0.0,45
"(Angel, Alexis)",0,0,0.0,45


The features above can be selected to fit a model to predict future connections. I have been successful applying Gaussian Naive Bayes algorithm for classification. However, which classifier to use might depend on the structure and complexity of a network.   



Reference: https://networkx.github.io/documentation/networkx-1.10/index.html